<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine_tuning_mixed/twadr_5_pubmed_bigtweet_mixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:05 --:--:--     0bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE html>'
100  9115  100  9115    0     0   1628      0  0:00:05  0:00:05 --:--:--  2230
curl: (23) Failed writing body (288 != 1211)


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
Unpacking objects: 100% (9/9), done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'TwADR-L.fold-5.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [52]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-5.validation.csv",header=None)
valid.head()

,0,1
0,13,Insomnia
1,13,awake in the middle of the night
2,57,hypertrophic scarring
3,85,tremor
4,94,anxiety


In [53]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-5.test.csv",header=None)
test.head()

,0,1
0,13,up all night
1,13,insomnia
2,93,shooting
3,133,night terrors
4,231,lucid dreamer


In [54]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine_tuning_language_model_pubmed_bigtweet_mixed.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [56]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [57]:
data_clas.add_test(test,label=0)


In [0]:
data_lm.save()
data_clas.save()

In [59]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 'of',
 'and',
 'in',
 'to',
 'a',
 'for',
 'with',
 'diclofenac',
 'was',
 'were',
 'on',
 'is',
 'rt',
 'sodium',
 'by',
 'that',
 'as',
 'at',
 'patients',
 'or',
 'this',
 'from',
 'drug',
 'you',
 'mg',
 'i',
 'be',
 'disease',
 'study',
 'group',
 'treatment',
 'are',
 'pain',
 'after',
 'an',
 'not',
 'amp',
 'it',
 'have',
 'release',
 'p',
 'gluten',
 'we',
 'than',
 'all',
 'free',
 'no',
 'effect',
 'compared',
 'new',
 'using',
 'effects',
 'groups',
 'more',
 'both',
 'two',
 'your',
 'our',
 'significantly',
 'time',
 'about',
 'between',
 'inflammatory',
 'its',
 'significant',
 'out',
 'these',
 'but',
 'h',
 'drugs',
 'day',
 'use',
 'anti',
 'voltaren',
 'my',
 'which',
 'used',
 'can',
 'potassium',
 'dose',
 'has',
 'results',
 'placebo',
 'do',
 's',
 'one',
 'what',
 'also',
 'glutenfree',
 'there',
 'efficacy',
 'nt',
 'celiac',
 'may',
 'days',
 'treated',
 'm',
 'd

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [62]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.232944,4.622821,0.251897,16:02


In [63]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.481788,4.277824,0.287491,16:07


In [64]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.163538,4.236582,0.290951,16:04


In [65]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.105597,4.205799,0.294401,16:02


In [0]:
learn.save_encoder('bigtweet_fold0_first')

In [67]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('bigtweet_fold0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.177532,6.712024,0.107692,00:22


In [68]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.318234,5.464606,0.146154,00:27


In [69]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.791121,4.840206,0.223077,01:11


In [70]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.538183,4.523071,0.230769,01:07
1,5.366568,4.410579,0.246154,01:10


In [71]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.169967,4.284807,0.292308,01:06
1,4.992842,4.193567,0.300000,01:07


In [72]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.786079,4.119456,0.292308,01:04
1,4.648281,4.032283,0.284615,01:08


In [73]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.467114,4.023465,0.253846,01:07
1,4.377900,3.901592,0.300000,01:06


In [74]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.211553,3.880765,0.323077,01:06
1,4.070609,3.820023,0.330769,01:07


In [75]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.949670,3.816614,0.323077,01:07
1,3.866464,3.877105,0.338462,01:07
2,3.711727,3.846458,0.330769,01:04
3,3.541283,3.806462,0.330769,01:11


In [76]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.402953,3.832185,0.346154,01:11
1,3.394854,3.938060,0.315385,01:05
2,3.235163,3.827006,0.330769,01:10
3,3.126205,3.834889,0.346154,01:07


In [77]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.982091,4.062484,0.284615,01:09
1,2.968877,4.014810,0.353846,01:07
2,2.879367,3.995340,0.353846,01:06
3,2.730465,3.961685,0.353846,01:04


In [78]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.669862,4.091197,0.307692,01:04
1,2.621618,4.253128,0.300000,01:08
2,2.509878,4.205885,0.330769,01:06
3,2.402874,4.125612,0.323077,01:04


In [79]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.328800,4.213937,0.346154,01:10
1,2.334131,4.215610,0.323077,01:08
2,2.248795,4.185946,0.330769,01:05
3,2.147209,4.203014,0.338462,01:07


In [80]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.140819,4.351876,0.307692,01:05
1,2.140825,4.387731,0.284615,01:04
2,2.055073,4.388662,0.307692,01:07
3,1.991027,4.366413,0.300000,01:10


In [81]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.948039,4.394938,0.323077,01:05
1,1.957628,4.557383,0.330769,01:10
2,1.933082,4.565267,0.338462,01:05
3,1.850699,4.471840,0.338462,01:08


In [82]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.736631,4.436610,0.323077,01:06
1,1.832541,4.703314,0.300000,01:05
2,1.828225,4.657432,0.338462,01:09
3,1.775587,4.574573,0.346154,01:05


In [83]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.815791,4.784319,0.330769,01:06
1,1.850941,4.870875,0.300000,01:11
2,1.801616,4.897393,0.323077,01:10
3,1.756496,4.861541,0.284615,01:07


In [84]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.753893,4.786546,0.338462,01:06
1,1.751240,4.871145,0.315385,01:12


In [85]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.732818,4.730281,0.338462,01:07
1,1.710170,4.868706,0.315385,01:10


In [0]:
# save the best model

learn.save_encoder('ask-0-tuning-bigtweet')

# Part three: Predict on the test dataset

In [87]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [88]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

143
143
[1099, 974, 1340, 133, 908, 233, 233, 249, 300, 1727, 945, 2191, 1340, 532, 550, 1373, 224, 749, 1340, 744, 744, 749, 749, 1597, 1597, 1597, 974, 792, 1884, 813, 224, 720, 1308, 1599, 1278, 858, 1340, 974, 974, 974, 978, 1340, 978, 978, 978, 1340, 1007, 1019, 1204, 949, 2147, 1348, 1925, 1136, 1610, 749, 1340, 561, 1295, 1227, 1340, 1340, 1340, 1308, 908, 1340, 1925, 908, 1325, 1340, 1340, 809, 1940, 1341, 1508, 1340, 224, 1340, 1340, 418, 749, 1340, 1340, 1356, 1597, 1597, 1340, 1610, 908, 1340, 1340, 749, 1340, 1048, 1521, 749, 1711, 1340, 1340, 1340, 1727, 1718, 2026, 1340, 1669, 1340, 1727, 1340, 1073, 233, 1787, 1099, 1718, 393, 1787, 1787, 1787, 923, 1815, 1815, 1973, 907, 2074, 1936, 1936, 224, 2149, 908, 1340, 1340, 858, 1940, 1787, 1787, 1340, 2081, 2103, 1529, 1787, 233, 2032, 1529, 1787]
[13, 13, 93, 133, 231, 233, 233, 249, 300, 362, 396, 418, 441, 532, 550, 582, 582, 596, 644, 744, 745, 749, 749, 791, 791, 791, 792, 792, 792, 813, 815, 818, 822, 852, 857, 858, 974,

In [89]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

41
0.2867132867132867
